<a href="https://colab.research.google.com/github/MohamedZAIM4/NLP_TPs/blob/main/Tp3_NLP_Classification_de_texte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
from nltk.corpus import stopwords
import string
import gensim
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from google.colab import files
import io
import pandas as pd

# Upload the file
uploaded = files.upload()

# Read the uploaded CSV file
csv_data = io.BytesIO(uploaded['movie_review.csv'])
df = pd.read_csv(csv_data)

Saving movie_review.csv to movie_review.csv


In [6]:
# Prétraitement des données textuelles
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    text = text.lower()  # Convertir en minuscules
    text = ''.join([char for char in text if char not in string.punctuation])  # Supprimer la ponctuation
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

In [7]:
df['preprocessed_text'] = df['text'].apply(preprocess_text)


In [8]:
# Entraînement du modèle Word2Vec
model = Word2Vec(df['preprocessed_text'], vector_size=100, window=5, min_count=1, workers=4)

In [9]:

# Vectorisation des reviews de movies
def vectorize_text(text):
    vectors = []
    for word in text:
        if word in model.wv:
            vectors.append(model.wv[word])
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return [0] * 100  # Vecteur de dimension 100 rempli de zéros si aucun mot n'est trouvé

In [10]:
df['vectorized_text'] = df['preprocessed_text'].apply(vectorize_text)


In [12]:
import numpy as np
# Division des données
X = np.array(df['vectorized_text'].to_list())
y = df['tag']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
# Construction d'un classificateur
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [15]:
# Évaluation du modèle
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

In [16]:
# Évaluation du modèle
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

In [17]:
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.5735475896168108
Precision: 0.5752354884483026
Recall: 0.5735475896168108
F1 score: 0.5685536920576343
